In [ ]:
'''
Before importing the libraries below, make sure they are installed. You can use Anaconda-Navigator's GUI or it's command line program called conda. You can also use the pip command.
Example install commands:

conda install pandas
pip install pandas

conda install beautifulsoup4
pip install beautifulsoup4
'''

# Import library to make HTTP requests, i.e fetch URL similar to a browser request
import requests
# Import web scraping library
from bs4 import BeautifulSoup
# Import data analysis and export library
import pandas as pd

In [ ]:
# Define a list of tickers to loop through and scrape for its historical pricing data.
tickers = ['SPY','IWO']

In [ ]:
# Initialize an empty Python dictionary to store the DataFrames for each ticker.
df_dict = {}

for ticker in tickers:
    
    # Request Yahoo Finance URL for a ticker's history (date range defaults to the past year).
    # https://finance.yahoo.com/quote/SPY/history

    yahoo_finance_url = 'https://finance.yahoo.com/quote/' + ticker + '/history'
    page = requests.get(yahoo_finance_url)

    # Assign HTML content to a BeautifulSoup object to facilitate web scraping.
    # html.parser is one of four parser libraries.
    # Description of each parser library: https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser
    soup = BeautifulSoup(page.content, 'html.parser')

    # Find all of the HTML table rows containing the historical data by looking for an HTML tab and its defining attribute.
    # You can inspect the HTML elements by using the Chrome browser and right clicking over the desired element and clicking "Inspect."
    # Raw HTML of the table row:
    # <tr class="BdT Bdc($c-fuji-grey-c) Ta(end) Fz(s) Whs(nw)">
    historical_prices = soup.select('tr[class="BdT Bdc($c-fuji-grey-c) Ta(end) Fz(s) Whs(nw)"]')

    # Create an empty Python dictionary data structure to keep track of the found price dates and the adjusted closing price for each row.
    # The value for each dictionary key is an empty list.
    # The number of items in the value list for the price_date and price_adj_close keys must be identical.
    # Dictionary initialization must be in the same cell to clear it out each time you're testing the scrape.
    # Data is appended and may lead to mismatched column counts.
    historical_data = {
        'price_date' : [],
        'price_adj_close' : []
    }

    # "for" loop through the found rows of historical data. Each row represents a single date.
    for historical_price in historical_prices:

        # display a horizontal line to easily distinguish between each row
        print("\n----------------\n")

        # Price Adj Close
        # Need to find price adj close first b/c not all rows have it due to a row with just the dividend.
        # <td class="Py(10px) Pstart(10px)"><span>278.25</span></td>
        # Find the HTML table cell tag (td) with the class value of "Py(10px) Pstart(10px)"
        rows = historical_price.select('td[class="Py(10px) Pstart(10px)"]')

        # Proceed if the HTML table cell tag is found with the class value defined above.
        if rows:
            # The following print statements are for debugging to show how the price adj close was found.
            # Price Adj Close is the 5th HTML table cell.
            # There are multiple HTML table cells with the class attribute value of "Py(10px) Pstart(10px)"
            #print(rows)
            print("price_adj_close HTML:",rows[4])
            # .text will exclude the HTML tags and only get the text
            # "\n" is a newline to format the debugging output
            print("price_adj_close text:",rows[4].text,"\n")
            price_adj_close = rows[4].text

            # Append the found price adj close to the price_adj_close dictionary key which we'll reference later when exporting the results.
            historical_data['price_adj_close'].append(price_adj_close)


            # Price Date
            # The price date has its own unique class value for the HTML table cell that it is in.
            # Raw HTML of the desired table cell:
            # <td class="Py(10px) Ta(start) Pend(10px)"><span>Mar 11, 2019</span></td>
            price_date = historical_price.select('td[class="Py(10px) Ta(start) Pend(10px)"]')

            # Proceed if a price date value is found.
            # Notice how the price date related code is indented inside the "if rows" statement.
            # This will skip over the rows with just a dividend value without all of the historical pricing data.
            if price_date:
                print("price_date HTML", price_date)
                print("price_date text:",price_date[0].text,"\n")
                price_date = price_date[0].text

                # Append the price date value to the price_date dictionary key.
                historical_data['price_date'].append(price_date)

    # This is why we created a Python dictionary to store the scraped results.
    # Assign the Python dictionary to a Pandas DataFrame.
    # By storing the data in a Pandas DataFrame we can manipulate the data then export the manipulated results.
    # df is a variable name for the DataFrame.
    df_dict[ticker] = pd.DataFrame(historical_data)

    # The price_adj_close column is seen as an object and not a float.
    # We want to convert price_adj_close to a float to allow for float operations within Excel.
    df_dict[ticker]['price_adj_close'] = df_dict[ticker]['price_adj_close'].astype(float)

    # price_date is listed as an object but will be treated as a string in Excel.
    # Convert price_date to a datetime format for date-related operations.
    df_dict[ticker]['price_date'] = pd.to_datetime(df_dict[ticker]['price_date'])

In [ ]:
# Ensure xlsxwriter module installed to create Excel xlsx files.
# https://xlsxwriter.readthedocs.io/example_pandas_multiple.html

excel_filename = 'historical_data.xlsx'

writer = pd.ExcelWriter(excel_filename, engine='xlsxwriter')

# Create a DataFrame of just the ticker symbols.
tickers_df = pd.DataFrame(tickers, columns=['Ticker'])
# Write the tickers DataFrame to its own sheet in the Excel file.
tickers_df.to_excel(writer, sheet_name='Tickers', index=False)

# Loop through the tickers.
for ticker in tickers:
    # Write the DataFrame for each ticker's historical prices to its own worksheet in the Excel file.
    # Use the ticker as the sheet name.
    df_dict[ticker].to_excel(writer, sheet_name=ticker, index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

# Open historical_data.xlsx in Excel to verify the export.
# historical_data.xlsx is written to the same directory as this notebook.